# Основы работы с Git

## Разрешение конфликтов

Когда несколько членов команды работают с репозиторием, то неизбежно будут возникать конфликты при синхронизации изменений.

### Первый разработчик

In [2]:
cd ~/repo

In [5]:
echo "Эта строка не вызывает конфликт" >> README.md

In [7]:
echo "0.2.0" > version.txt

In [8]:
git commit -am "Добавить коммит для демонстрации конфликта"

[master d1d8db0] Добавить коммит для демонстрации конфликта
 2 files changed, 2 insertions(+), 1 deletion(-)


In [10]:
git push origin HEAD:master

Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 8 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (10/10), 1011 bytes | 1011.00 KiB/s, done.
Total 10 (delta 4), reused 0 (delta 0), pack-reused 0
To /home/jovyan/remote/origin.git
   5299e1f..d1d8db0  HEAD -> master


### Второй разработчик

In [1]:
cd ../local

In [23]:
git checkout master

Switched to branch 'master'
Your branch is up to date with 'origin/master'.


In [24]:
echo "# Version" > version.txt
echo "0.2.1" >> version.txt

In [25]:
git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   version.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [26]:
git commit -am "Изменить версию на 0.2.1"

[master a4e2c40] Изменить версию на 0.2.1
 1 file changed, 2 insertions(+), 1 deletion(-)


In [11]:
commit=$(git log -n 1 --format="%H")

In [27]:
git fetch origin

remote: Enumerating objects: 13, done.        
remote: Counting objects: 100% (13/13), done.        
remote: Compressing objects: 100% (7/7), done.        
remote: Total 10 (delta 4), reused 0 (delta 0), pack-reused 0        
Unpacking objects: 100% (10/10), 991 bytes | 247.00 KiB/s, done.
From /home/jovyan/remote/origin
   5299e1f..d1d8db0  master     -> origin/master


In [29]:
git log -n 1 origin/master

commit d1d8db0c4d9c78dd133ccba48957be67713ca738 (origin/master)
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Sat Mar 18 08:00:37 2023 +0000

    Добавить коммит для демонстрации конфликта


In [32]:
git rebase origin/master || true

Auto-merging version.txt
CONFLICT (content): Merge conflict in version.txt
error: could not apply a4e2c40... Изменить версию на 0.2.1
hint: Resolve all conflicts manually, mark them as resolved with
hint: "git add/rm <conflicted_files>", then run "git rebase --continue".
hint: You can instead skip this commit: run "git rebase --skip".
hint: To abort and get back to the state before "git rebase", run "git rebase --abort".
Could not apply a4e2c40... Изменить версию на 0.2.1


In [33]:
git status

interactive rebase in progress; onto d1d8db0
Last command done (1 command done):
   pick a4e2c40 Изменить версию на 0.2.1
No commands remaining.
You are currently rebasing branch 'master' on 'd1d8db0'.
  (fix conflicts and then run "git rebase --continue")
  (use "git rebase --skip" to skip this patch)
  (use "git rebase --abort" to check out the original branch)

Unmerged paths:
  (use "git restore --staged <file>..." to unstage)
  (use "git add <file>..." to mark resolution)
	both modified:   version.txt

no changes added to commit (use "git add" and/or "git commit -a")


**Способ 0**: Самое простое - это остановить rebase, передав ключ `--abort` команде `git rebase`:

In [34]:
git rebase --abort

In [35]:
git status

On branch master
Your branch and 'origin/master' have diverged,
and have 1 and 3 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

nothing to commit, working tree clean


### Решение конфликта

In [39]:
git rebase origin/master || true

Auto-merging version.txt
CONFLICT (content): Merge conflict in version.txt
error: could not apply a4e2c40... Изменить версию на 0.2.1
hint: Resolve all conflicts manually, mark them as resolved with
hint: "git add/rm <conflicted_files>", then run "git rebase --continue".
hint: You can instead skip this commit: run "git rebase --skip".
hint: To abort and get back to the state before "git rebase", run "git rebase --abort".
Could not apply a4e2c40... Изменить версию на 0.2.1


In [40]:
git status

interactive rebase in progress; onto d1d8db0
Last command done (1 command done):
   pick a4e2c40 Изменить версию на 0.2.1
No commands remaining.
You are currently rebasing branch 'master' on 'd1d8db0'.
  (fix conflicts and then run "git rebase --continue")
  (use "git rebase --skip" to skip this patch)
  (use "git rebase --abort" to check out the original branch)

Unmerged paths:
  (use "git restore --staged <file>..." to unstage)
  (use "git add <file>..." to mark resolution)
	both modified:   version.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [41]:
cat version.txt

<<<<<<< HEAD
0.2.0
# Version
0.2.1
>>>>>>> a4e2c40 (Изменить версию на 0.2.1)


**Способ 1:** Применить изменения из `origin/master`

In [44]:
git checkout origin/master -- version.txt

In [45]:
git status

interactive rebase in progress; onto d1d8db0
Last command done (1 command done):
   pick a4e2c40 Изменить версию на 0.2.1
No commands remaining.
You are currently rebasing branch 'master' on 'd1d8db0'.
  (all conflicts fixed: run "git rebase --continue")

nothing to commit, working tree clean


In [46]:
git rebase --continue

Successfully rebased and updated refs/heads/master.                             


In [47]:
git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [48]:
git log -n 3

commit d1d8db0c4d9c78dd133ccba48957be67713ca738 (HEAD -> master, origin/master)
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Sat Mar 18 08:00:37 2023 +0000

    Добавить коммит для демонстрации конфликта

commit 0348b6978d51030ea92b89ee55a49dfb73fc8bf6
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Fri Mar 17 20:22:56 2023 +0000

    Добавить 42.txt

commit c963481a65554d3700813048e84751896619ea57
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Fri Mar 17 20:22:55 2023 +0000

    Добавить hello.txt


**Способ 2:** Применить локальные изменения

In [13]:
git reset --hard "$commit"

HEAD is now at a4e2c40 Изменить версию на 0.2.1


In [51]:
git log -n 1

commit a4e2c4075b1a868e1e2b40f00f83ea4a7c7205d8 (HEAD -> master)
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Sat Mar 18 08:05:40 2023 +0000

    Изменить версию на 0.2.1


In [53]:
git rebase origin/master || true

Auto-merging version.txt
CONFLICT (content): Merge conflict in version.txt
error: could not apply a4e2c40... Изменить версию на 0.2.1
hint: Resolve all conflicts manually, mark them as resolved with
hint: "git add/rm <conflicted_files>", then run "git rebase --continue".
hint: You can instead skip this commit: run "git rebase --skip".
hint: To abort and get back to the state before "git rebase", run "git rebase --abort".
Could not apply a4e2c40... Изменить версию на 0.2.1


In [54]:
git status

interactive rebase in progress; onto d1d8db0
Last command done (1 command done):
   pick a4e2c40 Изменить версию на 0.2.1
No commands remaining.
You are currently rebasing branch 'master' on 'd1d8db0'.
  (fix conflicts and then run "git rebase --continue")
  (use "git rebase --skip" to skip this patch)
  (use "git rebase --abort" to check out the original branch)

Unmerged paths:
  (use "git restore --staged <file>..." to unstage)
  (use "git add <file>..." to mark resolution)
	both modified:   version.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [55]:
git checkout master -- version.txt

In [56]:
git status

interactive rebase in progress; onto d1d8db0
Last command done (1 command done):
   pick a4e2c40 Изменить версию на 0.2.1
No commands remaining.
You are currently rebasing branch 'master' on 'd1d8db0'.
  (all conflicts fixed: run "git rebase --continue")

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   version.txt



In [2]:
GIT_EDITOR=true git rebase --continue

hint: Waiting for your editor to close the file...
[detached HEAD f59e4a7] Изменить версию на 0.2.1
 1 file changed, 2 insertions(+), 1 deletion(-)
Successfully rebased and updated refs/heads/master.                             


In [3]:
git log -n 1

commit f59e4a725284f6c6c6ec0af66003d360d2ed6d21 (HEAD -> master)
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Sat Mar 18 08:05:40 2023 +0000

    Изменить версию на 0.2.1


**Способ 3:** Ручное решение конфликта

In [18]:
git reset --hard "$commit"

HEAD is now at a4e2c40 Изменить версию на 0.2.1


In [19]:
git rebase origin/master || true

Auto-merging version.txt
CONFLICT (content): Merge conflict in version.txt
error: could not apply a4e2c40... Изменить версию на 0.2.1
hint: Resolve all conflicts manually, mark them as resolved with
hint: "git add/rm <conflicted_files>", then run "git rebase --continue".
hint: You can instead skip this commit: run "git rebase --skip".
hint: To abort and get back to the state before "git rebase", run "git rebase --abort".
Could not apply a4e2c40... Изменить версию на 0.2.1


In [20]:
git status

interactive rebase in progress; onto d1d8db0
Last command done (1 command done):
   pick a4e2c40 Изменить версию на 0.2.1
No commands remaining.
You are currently rebasing branch 'master' on 'd1d8db0'.
  (fix conflicts and then run "git rebase --continue")
  (use "git rebase --skip" to skip this patch)
  (use "git rebase --abort" to check out the original branch)

Unmerged paths:
  (use "git restore --staged <file>..." to unstage)
  (use "git add <file>..." to mark resolution)
	both modified:   version.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [21]:
cat version.txt

<<<<<<< HEAD
0.2.0
# Version
0.2.1
>>>>>>> a4e2c40 (Изменить версию на 0.2.1)


In [23]:
echo "#Version" > version.txt
echo "0.2.0" >> version.txt

In [24]:
git status

interactive rebase in progress; onto d1d8db0
Last command done (1 command done):
   pick a4e2c40 Изменить версию на 0.2.1
No commands remaining.
You are currently rebasing branch 'master' on 'd1d8db0'.
  (fix conflicts and then run "git rebase --continue")
  (use "git rebase --skip" to skip this patch)
  (use "git rebase --abort" to check out the original branch)

Unmerged paths:
  (use "git restore --staged <file>..." to unstage)
  (use "git add <file>..." to mark resolution)
	both modified:   version.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [25]:
git add version.txt

In [26]:
git status

interactive rebase in progress; onto d1d8db0
Last command done (1 command done):
   pick a4e2c40 Изменить версию на 0.2.1
No commands remaining.
You are currently rebasing branch 'master' on 'd1d8db0'.
  (all conflicts fixed: run "git rebase --continue")

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   version.txt



In [27]:
git diff --cached

diff --git a/version.txt b/version.txt
index 0ea3a94..0d701dc 100644
--- a/version.txt
+++ b/version.txt
@@ -1 +1,2 @@
+#Version
 0.2.0


In [28]:
GIT_EDITOR=true git rebase --continue

hint: Waiting for your editor to close the file...
[detached HEAD bea2e89] Изменить версию на 0.2.1
 1 file changed, 1 insertion(+)
Successfully rebased and updated refs/heads/master.                             


In [30]:
git log -n 3

commit bea2e8967ef4b52db5ba3f27f54ae777703c840e (HEAD -> master)
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Sat Mar 18 08:05:40 2023 +0000

    Изменить версию на 0.2.1

commit d1d8db0c4d9c78dd133ccba48957be67713ca738 (origin/master)
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Sat Mar 18 08:00:37 2023 +0000

    Добавить коммит для демонстрации конфликта

commit 0348b6978d51030ea92b89ee55a49dfb73fc8bf6
Author: Nikita Eshkeev <neshkeev@yandex.ru>
Date:   Fri Mar 17 20:22:56 2023 +0000

    Добавить 42.txt
